# Investigating claim actions

Exploring the data for claims/labels in order to understand whether we can identify the number of statements a given file might have based on edit comments.

In [4]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


## Configuration variables

In [2]:
wmf_snapshot = '2019-11'
start_date = '2019-01-01' # first date of caption edits
end_date = '2019-12-01' # last date of caption edits (exclusive)

# Investigating claim actions

I dug into the edit comments a bit to find examples, and found that [this file](https://commons.wikimedia.org/wiki/File:Rosendahl,_Darfeld,_Ortsansicht_--_2014_--_9391.jpg) has a bunch of property edits that provided good insight into what to look for. Basically, it meant that I had to expand my search to anything that starts with "wb". Using that insight, the below query looks for an edit comment matching "wb{something}-{something}:" (anchored at the start of the comment with space for "/* ") and aggregates over the first and second "something".

In [18]:
claim_query = '''
SELECT claim_action, claim_subaction, count(*) AS num_actions
FROM (
    SELECT
        regexp_extract(event_comment, "^...(wb[^-]+)", 1) AS claim_action,
        regexp_extract(event_comment, "^...wb[^-]+-([^:]+):", 1) AS claim_subaction
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND wiki_db = "commonswiki"
    AND event_entity = "revision"
    AND event_type = "create"
    AND event_timestamp >= "{start_date}"
    AND event_timestamp < "{end_date}"
    AND page_is_deleted = false -- only count live pages
    AND page_namespace = 6 -- only count files
    AND event_comment REGEXP "^...(wb[^-]+)-([^:]+):"
) AS ce
GROUP BY claim_action, claim_subaction
'''

In [19]:
claim_counts = hive.run(
    [
        "SET mapreduce.map.memory.mb=4096", 
        claim_query.format(
            snapshot = wmf_snapshot,
            start_date = start_date,
            end_date = end_date
        )
    ]
)

In [28]:
claim_counts.sort_values(['claim_action', 'claim_subaction'])

,claim_action,claim_subaction,num_actions
2,wbcreateclaim,create,1655105
8,wbeditentity,update,261794
0,wbeditentity,update-languages,872
11,wbremoveclaims,remove,22021
3,wbremoveclaims,update,14259
5,wbsetclaim,create,537587
6,wbsetclaim,update,40832
9,wbsetdescription,add,2
10,wbsetlabel,add,1631793
7,wbsetlabel,remove,11321


Based on this, I need to ask the developers what the various edit comments are and what they mean. Also, we know from the example file that a single edit can modify multiple properties, meaning that we cannot know how many properties a file has based on the comments.